In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cellc = pd.read_csv("/data/Genetic/data/EstimateCellCounts_PPMI_EPICn524final030618.csv")
pat_stat = pd.read_csv("/data/Genetic/data/Patient_Status.csv")
pat_info = pd.read_csv("/data/Genetic/data/PPMI_Meth_n524_for_LONI030718.tsv", delimiter='\t')
raw_dat = pd.read_csv("/data/Genetic/data/beta_post_Funnorm_PPMI_EPICn524final030618.csv")

In [ ]:
pat_numbers = list(pat_info["PATNO"].astype(int))

In [ ]:
is_PD = lambda id: pat_stat[pat_stat.PATNO == id]["RECRUITMENT_CAT"].values[0] == 'PD'
get_sentrix = lambda id: str(pat_info[pat_info.PATNO == id]["Sentrix ID"].values[0]) + "_" + pat_info[pat_info.PATNO == id]["Sentrix Position"].values[0]
getXVal_cellc = lambda id: cellc[cellc.Sentrix_position == get_sentrix(id)].values[0][1:].astype(float)
getXVal_raw = lambda id: raw_dat[get_sentrix(id)].values

In [ ]:
X = np.zeros((len(pat_numbers), 864067))
y = np.zeros((len(pat_numbers)), dtype=int)

In [ ]:
for i, number in enumerate(pat_numbers):
    if(is_PD(number)):
        y[i] = 1
    X[i] = getXVal_raw(number)
raw_dat = None
import gc
gc.collect()

In [ ]:
from sklearn.decomposition import PCA as pca
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split as splt
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import matthews_corrcoef as mcc

forest = rfc(n_estimators = 1000, n_jobs = -1)
decomp = pca(n_components=32, svd_solver='full')
X_tr, X_ts, y_tr, y_ts = splt(X, y, test_size = 0.2, random_state = 42, stratify = y)
decomp.fit(X_tr)
X_tr_decomp = decomp.transform(X_tr)
X_ts_decomp = decomp.transform(X_ts)
forest.fit(X_tr_decomp, y_tr)
y_ts_pred = forest.predict(X_ts_decomp)

In [ ]:
acc(y_ts, y_ts_pred), mcc(y_ts, y_ts_pred)

In [ ]:
decomp2 = pca(n_components=2, svd_solver='full')

In [ ]:
decomp2.fit(X_tr)
X_decomp = decomp2.transform(X)
y

In [ ]:
x1 = []
x0 = []
for i, x in enumerate(X_decomp):
    if y[i] == 1:
        x1.append(x)
    else:
        x0.append(x)
x1 = np.array(x1)
x0 = np.array(x0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(x1[:,0], x1[:,1], 'ro')
plt.plot(x0[:,0], x0[:,1], 'bo')
plt.show()